<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Helium_Mobius_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp

# =============================
# Variáveis simbólicas
# =============================
u,v,w,zeta = sp.symbols('u v w zeta', positive=True)

# Coordenadas perimétricas → r1, r2, r12
r1 = (u+v)/2
r2 = (u+w)/2
r12= (v+w)/2

# Jacobiano
J = sp.Rational(1,8)*u*v*w*(u+v)*(u+w)*(v+w)

# =============================
# Base de Hylleraas até N=2
# =============================
def hylleraas_basis(N=2):
    B=[]
    for i in range(N+1):
        for j in range(N+1-i):
            for k in range(N+1-i-j):
                B.append((i,j,k))
    return B

basis = hylleraas_basis(N=2)
print("Base Hylleraas N=2 =", basis)

# =============================
# Função φ_{i,j,k}
# =============================
def phi(i,j,k):
    return (r1**i)*(r2**j)*(r12**k)*sp.exp(-zeta*(r1+r2))

# ============================================================
# Função que calcula a integral ∫ φ_a φ_b J du dv dw analiticamente
# ============================================================
def integral_moment(expr):
    """
    expr deve ser algo do tipo (polinómio)*exp(-2ζu - ζv - ζw).
    Retiramos explicitamente exp(...) para expandir o polinómio.
    """
    # 1) fator exponencial conhecido
    expo = sp.exp(-2*zeta*u - zeta*v - zeta*w)
    poly_part = sp.simplify(expr / expo)

    # 2) expandimos só o polinómio
    poly_expanded = sp.expand(poly_part)

    res = 0
    for term in poly_expanded.as_ordered_terms():
        coeff = term.as_coeff_Mul()[0]
        mon   = term/coeff

        # expoentes de u,v,w
        a = sp.degree(mon, u)
        b = sp.degree(mon, v)
        c = sp.degree(mon, w)

        # termo integral:
        # ∫ u^a e^{-2ζu} du = a! / (2ζ)^{a+1}, etc.
        res_term = (
            coeff
            * sp.factorial(a)/(2*zeta)**(a+1)
            * sp.factorial(b)/(zeta)**(b+1)
            * sp.factorial(c)/(zeta)**(c+1)
        )
        res += res_term

    return sp.simplify(res)

# =============================
# Construção simbólica da matriz S
# =============================
K = len(basis)
S = sp.zeros(K,K)

for a,(i,j,k) in enumerate(basis):
    for b,(p,q,r) in enumerate(basis):
        expr = phi(i,j,k)*phi(p,q,r)*J
        S[a,b] = integral_moment(expr)

print("\nMatriz de sobreposição S (analítica):")
sp.pprint(S)


Base Hylleraas N=2 = [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0), (0, 1, 1), (0, 2, 0), (1, 0, 0), (1, 0, 1), (1, 1, 0), (2, 0, 0)]

Matriz de sobreposição S (analítica):
⎡ 25      159      591      225      795      153      225      795      513   ↪
⎢─────   ──────   ──────   ──────   ──────   ──────   ──────   ──────   ────── ↪
⎢    9       10       11       10       11       11       10       11       11 ↪
⎢16⋅ζ    32⋅ζ     32⋅ζ     64⋅ζ     64⋅ζ     16⋅ζ     64⋅ζ     64⋅ζ     64⋅ζ   ↪
⎢                                                                              ↪
⎢ 159     591      5019     795      6501     1197     795      6501     3957  ↪
⎢──────  ──────   ──────   ──────   ───────  ──────   ──────   ───────  ────── ↪
⎢    10      11       12       11        12      12       11        12       1 ↪
⎢32⋅ζ    32⋅ζ     64⋅ζ     64⋅ζ     128⋅ζ    32⋅ζ     64⋅ζ     128⋅ζ    128⋅ζ  ↪
⎢                                                                              ↪
⎢ 591     5019     